In [1]:
import os

In [2]:
%pwd

'/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project/kidney-disease-classification-using-mlflow--End-to-End-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/user/Desktop/data science/kidney-disease-classification-using-mlflow--End-to-End-Project/kidney-disease-classification-using-mlflow--End-to-End-Project'

In [5]:
os.environ['MLFLOW_TRACKING_URL']="https://dagshub.com/shameemmon.mk/kidney-disease-classification-using-mlflow--End-to-End-Project.mlflow"
os.environ['MLFLOW_TRACKING_URL']='shameemmon.mk'
import dagshub
dagshub.init(repo_owner='shameemmon.mk', repo_name='kidney-disease-classification-using-mlflow--End-to-End-Project', mlflow=True)

Accessing as shameemmon.mk

Initialized MLflow to track repo "shameemmon.mk/kidney-disease-classification-using-mlflow--End-to-End-Project"

Repository shameemmon.mk/kidney-disease-classification-using-mlflow--End-to-End-Project initialized!

In [6]:
import tensorflow as tf 

2024-08-31 22:07:37.699923: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-31 22:07:37.745887: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-31 22:07:37.746714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-31 22:07:38.433850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

OSError: No file or directory found at artifacts/training/model.h5

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model : Path
    training_data : Path 
    all_params : dict
    mlflow_url:str
    #mlflow_params: str
    params_image_size : list
    params_batch_size : int




In [ ]:
from kidneyDisease.constants import *
from kidneyDisease.utils.common import read_yaml,create_directories,save_json

In [ ]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/trainig/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/shameemmon.mk/kidney-disease-classification-using-mlflow--End-to-End-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [ ]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [ ]:
class Evaluation:
    def __init__(self,config:EvaluationConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path:Path) -> tf.keras.Model:
             return tf.keras.models.load_model(path)
        

    def evaluation(self):
             self.model = self.load_model(self.config.path_of_model)
             self._valid_generator()
             self.score = model.evaluate(self.valid_generator)


    def save_score(self):
             score = {"loss":self.score[0],'accuracy':self.score[1]}
             save_json(path= Path ('score.json'),data=score)

        
    
    
    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            
            if tracking_url_type_store != "file":

            
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")
    
      




In [ ]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-08-31 16:58:37,933: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-08-31 16:58:37,937: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-31 16:58:37,938: INFO: common: created directory at: artifacts]


OSError: No file or directory found at artifacts/training/model.h5